# Content Chunk Ingestion

This notebook computes checksums for content chunks and ingests them into the `content_chunks` table.

## Workflow

1. **Load Content**: Load content metadata from `content_repo` table
2. **Read Markdown**: Read markdown files from disk
3. **Chunk Content**: Split content into fixed-size chunks (1000 characters)
4. **Compute Checksums**: Calculate SHA-256 hash for each chunk
5. **Ingest to Database**: Insert into `content_chunks` table with FK to `content_repo`

## Prerequisites

- Database schema must be created (`content_repo` and `content_chunks` tables)
- Content repository must be populated (run `0_content_repo_ingestion.ipynb` first)
- Markdown files must exist on disk

## Design Principles

- Simple: Character-based chunking, no complex algorithms
- Deterministic: Same content always produces same checksums
- Traceable: Each chunk links back to source file via `ud_source_file_id`
- Database-aware: Uses foreign keys to link chunks to files

## Setup and Configuration

In [ ]:
import sys
import sqlite3
from pathlib import Path
import pandas as pd

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

# Import modules
from granular_impact.chunking import SimpleChunker
from granular_impact.data_ingestion import ChunkIngestion

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

print("✓ Imports successful")

In [2]:
# Configuration
PROJECT_ROOT = Path.cwd().parent if 'granular_impact' in str(Path.cwd()) else Path.cwd()
DB_PATH = PROJECT_ROOT / "databases" / "my_database.db"

# Chunking configuration
CHUNK_SIZE = 1000  # Characters per chunk

print(f"Project Root: {PROJECT_ROOT}")
print(f"Database: {DB_PATH}")
print(f"Chunk Size: {CHUNK_SIZE} characters")

# Verify paths
assert DB_PATH.exists(), f"Database not found: {DB_PATH}"

print("\n✓ All paths verified")

Project Root: c:\Users\praka\projects\FAQ\FAQ_update
Database: c:\Users\praka\projects\FAQ\FAQ_update\databases\my_database.db
Chunk Size: 1000 characters

✓ All paths verified


---
## Step 1: Load Content Repository Data

Load content metadata from `content_repo` table to get list of files to process.

In [ ]:
print(f"{'='*80}")
print("  LOADING CONTENT REPOSITORY DATA")
print(f"{'='*80}\n")

# Load content_repo data
conn = sqlite3.connect(str(DB_PATH))
query = """
SELECT 
    ud_source_file_id,
    raw_file_nme,
    raw_file_type,
    title_nme,
    extracted_markdown_file_path,
    file_status
FROM content_repo
WHERE file_status = 'Active'
ORDER BY raw_file_nme
"""

content_df = pd.read_sql_query(query, conn)
conn.close()

print(f"Loaded {len(content_df)} active content files\n")
print(content_df.head())

print(f"\n✓ Content repository data loaded")

---
## Step 2: Chunk Content and Compute Checksums

For each file:
1. Read markdown content from disk
2. Split into fixed-size chunks
3. Compute SHA-256 checksum for each chunk

In [ ]:
print(f"\n{'='*80}")
print("  CHUNKING AND CHECKSUM COMPUTATION")
print(f"{'='*80}\n")

# Initialize chunker
chunker = SimpleChunker(chunk_size=CHUNK_SIZE)

# Process each file
files_processed = 0
total_chunks = 0
ingestion_results = []

# Initialize chunk ingestion
chunk_ingestion = ChunkIngestion(str(DB_PATH))

for idx, row in content_df.iterrows():
    # The path in the database is relative to PROJECT_ROOT
    markdown_path = PROJECT_ROOT / row['extracted_markdown_file_path']
    
    if not markdown_path.exists():
        print(f"⚠️  WARNING: File not found: {markdown_path}")
        continue
    
    # Read content
    try:
        content = markdown_path.read_text(encoding='utf-8')
    except Exception as e:
        print(f"✗ ERROR reading {markdown_path}: {e}")
        continue
    
    # Chunk and compute checksums
    chunks_with_checksums = chunker.chunk_with_checksums(content)
    
    # Ingest chunks for this file
    result = chunk_ingestion.ingest_chunks_for_file(
        ud_source_file_id=row['ud_source_file_id'],
        chunks_with_checksums=chunks_with_checksums,
        clear_existing=True  # Clear old chunks before re-ingestion
    )
    
    if result['success']:
        files_processed += 1
        total_chunks += result['rows_inserted']
        print(f"  ✓ {row['raw_file_nme']:40s} → {result['rows_inserted']:3d} chunks")
    else:
        print(f"  ✗ {row['raw_file_nme']:40s} → ERROR: {result['message']}")
    
    ingestion_results.append({
        'file_id': row['ud_source_file_id'],
        'file_name': row['raw_file_nme'],
        'success': result['success'],
        'chunks': result['rows_inserted']
    })

print(f"\n✓ Processed {files_processed} files")
print(f"✓ Generated {total_chunks} chunks")

In [ ]:
# Summary of ingestion results
results_df = pd.DataFrame(ingestion_results)

print(f"\nIngestion Results Summary:")
print(results_df.to_string(index=False))

print(f"\nData Summary:")
print(f"  Files processed: {results_df['success'].sum()}")
print(f"  Total chunks: {results_df['chunks'].sum()}")
print(f"  Failed files: {(~results_df['success']).sum()}")

---
## Step 3: Verify Ingestion

Verify the ingested chunks with sample queries.

In [ ]:
print(f"\n{'='*80}")
print("  DATABASE VERIFICATION")
print(f"{'='*80}\n")

# Get statistics
stats = chunk_ingestion.get_stats()

print("Database stats:")
for key, value in stats.items():
    print(f"  {key:30s}: {value}")

In [ ]:
# No longer needed - removed this cell

---
## Verification Queries

Query the database to verify ingested chunks.

In [ ]:
print(f"\n{'='*80}")
print("  VERIFICATION QUERIES")
print(f"{'='*80}\n")

conn = sqlite3.connect(str(DB_PATH))

# Query 1: Sample chunks
print("Query 1: Sample chunks from database\n")
query = """
SELECT
    chunk_id,
    ud_source_file_id,
    chunk_index,
    content_checksum,
    SUBSTR(chunk_text, 1, 50) || '...' AS chunk_text_preview,
    created_at,
    status
FROM content_chunks
WHERE status = 'active'
LIMIT 5
"""
result_df = pd.read_sql_query(query, conn)
print(result_df.to_string(index=False))

In [ ]:
# Query 2: Chunks per file with file info
print("\n\nQuery 2: Chunks per source file\n")
query = """
SELECT 
    cr.raw_file_nme,
    cr.title_nme,
    COUNT(cc.chunk_id) AS chunk_count,
    cr.file_status
FROM content_repo cr
LEFT JOIN content_chunks cc ON cr.ud_source_file_id = cc.ud_source_file_id
WHERE cc.status = 'active' OR cc.status IS NULL
GROUP BY cr.ud_source_file_id, cr.raw_file_nme, cr.title_nme, cr.file_status
ORDER BY chunk_count DESC
"""
result_df = pd.read_sql_query(query, conn)
print(result_df.to_string(index=False))

conn.close()

---
## Summary

### Completed:
✓ Loaded content metadata from `content_repo` table  
✓ Read markdown files from disk  
✓ Split content into fixed-size chunks (1000 characters)  
✓ Computed SHA-256 checksums for each chunk  
✓ Ingested chunks into `content_chunks` table with FK to `content_repo`  
✓ Verified data in database  

### Key Changes from Previous Version:
- ✅ Uses `content_chunks` table (not `content_checksums`)
- ✅ Links chunks to source files via `ud_source_file_id` FK
- ✅ Supports multiple file versions (same file, different versions)
- ✅ Enables precise change detection per file version

### Next Steps:
1. **Generate FAQs**: Run `2_faq_generation.ipynb` to generate FAQs from chunks
2. **Link Sources**: FAQs will reference chunks in `faq_question_sources` and `faq_answer_sources`
3. **Change Detection**: When content changes, new chunks enable precise change tracking
4. **Impact Analysis**: Identify which FAQs are affected by specific content changes

### Architecture Benefits:
- Each chunk belongs to a specific file version (via FK)
- Same content can exist in multiple files (shared boilerplate)
- Same content can exist at different positions (headers/footers)
- Change detection works at chunk level, not file level
- Supports incremental updates without re-processing entire corpus

In [ ]:
print(f"\n{'='*80}")
print("  CHUNK INGESTION COMPLETE")
print(f"{'='*80}\n")
print("✓ All chunks successfully computed and ingested")
print("✓ Data verified in database")
print("✓ Ready for FAQ generation")
print(f"\n{'='*80}")